---
title: "Juliaを用いたランベール問題のソルバーに関するノートブック" \
author: "Naoya Ozaki" \
date: "04 June 2022" \
output: "lambert_problem_via_julia"

---

# Juliaを用いたランベール問題のソルバーに関するノートブック
数値計算分野で用いられるJuliaを用いてランベール問題(Lambert's Problem)の解法を実装してみよう．

# 1. はじめに
ランベール問題とは，二体問題の力学環境下において，初期位置$\boldsymbol{r}_0$と終端位置$\boldsymbol{r}_\textrm{f}$と遷移時間TOF($=t_f-t_0$)が与えられた時に，初期速度$\boldsymbol{v}_0$と終端速度$\boldsymbol{v}_\textrm{f}$を解く問題であり，二点境界値問題の一種である．与えられた$\boldsymbol{r}_0$, $\boldsymbol{r}_\textrm{f}$, TOFを満たす解は複数存在する（＝一意に定まらない）ため，複数の解が出力される必要がある．`pykep`等でランベール問題用ソルバーが提供されているが，ここではゼロから実装することを考える．本実装を進める上で参考となる論文を以下に挙げる．

### 参考文献・論文
1. Dario Izzo, "Revisiting Lambert's problem", Celestial Mechanics and Dynamical Astoronomy, 2015, Vol 121, pp 1-15.
2. R. H. Battin, "An Introduction to the Mathematics and Methods of Astrodynamics, Revised Version", 1999, AIAA Education Series.
3. PyKEP (https://esa.github.io/pykep/) access on Nov.2017.
4. Fortran-Astrodynamics-Toolkit(https://github.com/jacobwilliams/Fortran-Astrodynamics-Toolkit) access on Nov.2017.

### Julia用のNAIF SPICE Toolkitのインストール
以下のGithubからインストールすることが可能である．\
https://github.com/JuliaAstro/SPICE.jl

参考情報：\
Acton, C.H.; "Ancillary Data Services of NASA's Navigation and Ancillary Information Facility;" Planetary and Space Science, Vol. 44, No. 1, pp. 65-70, 1996.

In [2]:
import Pkg; Pkg.add("SPICE")

   Updating registry at `~/.julia/registries/General`


###                                                                        4.5%

##############################                                            42.4%

########################################################                  78.3%

######################################################################## 100.0%


...

Updating `~/.julia/environments/v1.5/Manifest.toml`
  [79e6a3ab] 

↑ Adapt v2.3.0 ⇒ v3.3.3
  [0dad84c5] + ArgTools v1.1.1
  [07f52509] + CSPICE_jll v66.1.0+0
  [d360d2e6] + ChainRulesCore v1.12.2
  [9e997f8a] + ChangesOfVariables v0.1.3
  [35d6a980] ↑ ColorSchemes v3.10.2 ⇒ v3.18.0
  [3da002f7] ↑ ColorTypes v0.10.9 ⇒ v0.11.3
  [c3611d14] + ColorVectorSpace v0.9.9
  [5ae59095] ↑ Colors v0.12.4 ⇒ v0.12.8
  [34da2185] ↑ Compat v3.23.0 ⇒ v3.45.0
  [8f4d0f93] ↑ Conda v1.5.0 ⇒ v1.7.0
  [9a962f9c] ↑ DataAPI v1.4.0 ⇒ v1.10.0
  [864edb3b] ↑ DataStructures v0.18.8 ⇒ v0.18.13
  [ffbed154] + DocStringExtensions v0.8.6
  [f43a241f] + Downloads v1.6.0
  [c87230d0] ↑ FFMPEG v0.4.0 ⇒ v0.4.1
  [0656b61e] ↑ GLFW_jll v3.3.2+1 ⇒ v3.3.4+0
  [d2c73de3] ↑ GR_jll v0.53.0+0 ⇒ v0.58.1+0
  [5c1252a2] ↑ GeometryBasics v0.3.4 ⇒ v0.3.13
  [42e2da0e] ↑ Grisu v1.0.0 ⇒ v1.0.2
  [83e8ac13] ↑ IniFile v0.5.0 ⇒ v0.5.1
  [3587e190] + InverseFunctions v0.1.4
  [92d709cd] + IrrationalConstants v0.1.1
  [c8e1da08] ↑ IterTools v1.3.0 ⇒ v1.4.0
  [692b3bcd] ↑ JLLWrappers v1.1.3 ⇒ v1.4.1
  [682c

  [efe28fd5] + OpenSpecFun_jll v0.5.3+4
  [bac558e1] ↑ OrderedCollections v1.3.2 ⇒ v1.4.1
  [69de0a69] ↑ Parsers v1.0.14 ⇒ v2.3.1
  [ccf2f8ad] ↑ PlotThemes v2.0.0 ⇒ v2.0.1
  [995b91a9] ↑ PlotUtils v1.0.8 ⇒ v1.0.15
  [21216c6a] + Preferences v1.3.0
  [438e738f] ↑ PyCall v1.92.1 ⇒ v1.93.1
  [ea2cea3b] + Qt5Base_jll v5.15.2+0
  [ede63266] - Qt_jll v5.15.2+0
  [3cdcf5f2] ↑ RecipesBase v1.1.1 ⇒ v1.2.1
  [ae029012] ↑ Requires v1.1.1 ⇒ v1.3.0
  [5bab7191] + SPICE v0.2.2
  [6c6a2e73] ↑ Scratch v1.0.3 ⇒ v1.1.0
  [a2af1166] ↑ SortingAlgorithms v0.3.1 ⇒ v1.0.1
  [276daf66] + SpecialFunctions v2.1.6
  [90137ffa] ↑ StaticArrays v1.0.1 ⇒ v1.3.6
  [82ae8749] + StatsAPI v1.3.0
  [2913bbd2] ↑ StatsBase v0.33.2 ⇒ v0.33.16
  [09ab397b] ↑ StructArrays v0.4.4 ⇒ v0.6.8
  [fa267f1f] + TOML v1.0.3
  [3783bdb8] ↑ TableTraits v1.0.0 ⇒ v1.0.1
  [bd369af6] ↑ Tables v1.2.1 ⇒ v1.7.0
  [62fd8b95] + TensorCore v0.1.1
  [81def892] ↑ VersionParsing v1.2.0 ⇒ v1.3.0
  [2381bf8a] ↑ Wayland_protocols_jll v1.18.0+4 ⇒ v1.25.

   Building Conda ─→ `~/.julia/packages/Conda/x2UxR/deps/build.log`


   Building PyCall → `~/.julia/packages/PyCall/7a7w0/deps/build.log`


In [60]:
import SPICE

# Load generic kernels
SPICE.furnsh("/Users/naoya/Google Drive/21_Programming/SPICE/generic_kernels/lsk/naif0012.tls") # Leap seconds kernel
SPICE.furnsh("/Users/naoya/Google Drive/21_Programming/SPICE/generic_kernels/pck/gm_de431.tpc") # Gravity Constant
SPICE.furnsh("/Users/naoya/Google Drive/21_Programming/SPICE/generic_kernels/spk/planets/de440.bsp") # Planetary ephemeris kernel

# Convert the calendar date to ephemeris seconds past J2000
et = SPICE.utc2et("2018-02-06T20:45:00")

# Get the position of Mars at `et` w.r.t. Earth
SPICE.spkpos("mars_barycenter", et, "J2000", "none", "earth")

([-9.561252750118901e7, -2.045743839170729e8, -8.601309297738348e7], 806.0297773638883)

# 2. 理論編
TBW

# 3. 実装編

## 3.1. 前提条件
以下に示す前提条件でランベール問題の実装を進める．

In [72]:
import SPICE
# 重力定数
μ = SPICE.bodvrd("SUN", "GM", 1)[1];

# 飛行時間
et1 = SPICE.str2et("2031/03/01 00:00:00 UTC") # 地球出発日時, UTC
et2 = SPICE.str2et("2032/01/01 00:00:00 UTC") # 火星到着日時, UTC
tof = et2 - et1 # 飛行時間

#　地球・火星軌道
state_earth, _ = SPICE.spkez(399, et1, "ECLIPJ2000", "NONE", 10)
state_mars, _ = SPICE.spkez(4, et2, "ECLIPJ2000", "NONE", 10)

r1 = state_earth[1:3]
r2 = state_mars[1:3]

3-element Array{Float64,1}:
  2.0803122655020934e8
  1.8008145720647745e7
 -4.721309430616008e6

## 3.2. ランベール問題のソルバー
ランベール問題を解く際に利用される変数
$$
\begin{align}
c &= \|\boldsymbol{r}_2 - \boldsymbol{r}_1\| \\
s &= \frac{1}{2}\left(r_1 + r_2 + c\right) \\
\lambda^2 &= 1 - \frac{c}{s}
\end{align}
$$
および軌道を規定する基底を計算する．


In [73]:
using LinearAlgebra

# ランベール問題で用いられる幾何学的な変数
r1_norm = norm(r1)
r2_norm = norm(r2)
c = norm(r2-r1)
s = 0.5*(r1_norm + r2_norm + c)
λ2 = 1.0 - c_norm/s

# 無次元化時刻の計算
T = sqrt(2.0*μ/(s^3))*tof

3-element Array{Float64,1}:
 -0.018349153508173635
 -0.049835784657667
 -0.9985888559027125

In [75]:
# 基底ベクトル
ivec_r1 = r1/r1_norm
ivec_r2 = r2/r2_norm
ivec_h = cross(ivec_r1, ivec_r2)
ivec_h = ivec_h/norm(ivec_h) # 角運動量ベクトル
# NOTE: r1ベクトルとr2ベクトルが平行な場合，ivec_hが定義できないため，注意が必要．

if ivec_h[3] < 0.0
    λ = -sqrt(λ2)
    ivec_t1 = -cross(ivec_h, ivec_r1)
    ivec_t2 = -cross(ivec_h, ivec_r2)
else
    λ = sqrt(λ2)
    ivec_t1 = cross(ivec_h, ivec_r1)
    ivec_t2 = cross(ivec_h, ivec_r2)
end

3-element Array{Float64,1}:
 -0.0872249314162494
  0.9950288911397668
 -0.04805535492114194

In [76]:
# x, yの解法